In [ ]:
import numpy as np
# from scipy import signal
import scipy.io as sio
import matplotlib.pyplot as plt
import os
import sys
from typing import Dict
import glob

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

%load_ext autoreload
%autoreload 2

from data_preprocessing import experiments, process_single_trial_daily_activity

In [ ]:
data_path = os.getcwd() + "/../S2/raw/"
feat_path = os.getcwd() + "/../S2/features/"

subject = 2
experiments = [0,1,2,3,4,5]

In [ ]:
feature_data = {}
feature_data['emgFeature_disc'] = np.asarray([])
feature_data['accFeature_disc'] = np.asarray([])
feature_data['forceFeature_disc'] = np.asarray([])
feature_data['trialLabel_disc'] = []
feature_data['subGestureLabel_disc'] = []

feature_data['emgFeature_cont'] = np.asarray([])
feature_data['accFeature_cont'] = np.asarray([])
feature_data['forceFeature_cont'] = np.asarray([])
feature_data['trialLabel_cont'] = []
feature_data['subTrialLabel_cont'] = []

In [ ]:
for exp in experiments:
    fileHead = 'S' + f'{subject:03}' + '_E' + f'{exp:03}*'
    filenames = glob.glob(data_path + fileHead)
    disc_count = 0
    cont_count = 0
    
    for file in filenames:
        print(file)
        data = sio.loadmat(file)

        for k in data.keys():
            if k  not in ['raw', 'crc', '__header__', '__version__', '__globals__']:
                data[k] = data[k][0][0]

        emgFeat, accFeat, forceFeat, trialLabel, activityGestureLabels = process_single_trial_daily_activity(data, dropSamples=10, featureWindow = 50, log=False)

        if data['discrete']:
            print(f"disc trial: {disc_count}")
            if np.size(feature_data['emgFeature_disc']) == 0:
                print(f"discrete feature data empty, setting equal")
                feature_data['emgFeature_disc'] = emgFeat
                feature_data['accFeature_disc'] = accFeat
                feature_data['forceFeature_disc'] = forceFeat
                feature_data['trialLabel_disc'] = trialLabel + disc_count
                feature_data['subGestureLabel_disc'] = activityGestureLabels
            else:
                print(f"shape of discrete trial: {np.shape(emgFeat)}, shape of feature_data[emg] before appending: {np.shape(feature_data['emgFeature_disc'])}")
                feature_data['emgFeature_disc'] = np.concatenate((feature_data['emgFeature_disc'], emgFeat), axis=0)
                print(f"shape after appending: {np.shape(feature_data['emgFeature_disc'])}")
                feature_data['accFeature_disc'] = np.concatenate((feature_data['accFeature_disc'], accFeat), axis = 0)
                feature_data['forceFeature_disc'] = np.concatenate((feature_data['forceFeature_disc'], forceFeat), axis = 0)
                feature_data['trialLabel_disc'] = np.append(feature_data['trialLabel_disc'], trialLabel + disc_count)
                feature_data['subGestureLabel_disc'] = np.append(feature_data['subGestureLabel_disc'], activityGestureLabels)
            disc_count += 1
        elif data['continuous']:
            print(f"continuous trial: {cont_count}")
            if np.size(feature_data['emgFeature_cont']) == 0:
                print(f"continuous feature data empty, setting equal")
                feature_data['emgFeature_cont'] = emgFeat
                feature_data['accFeature_cont'] = accFeat
                feature_data['forceFeature_cont'] = forceFeat
                feature_data['trialLabel_cont'] = trialLabel + disc_count
                feature_data['subTrialLabel_cont'] = activityGestureLabels
            else :
                print(f"shape of continuous trial: {np.shape(emgFeat)}, shape of feature_data[emg] before appending: {np.shape(feature_data['emgFeature_cont'])}")
                feature_data['emgFeature_cont'] = np.concatenate((feature_data['emgFeature_cont'], emgFeat), axis=0)
                print(f"shape after appending: {np.shape(feature_data['emgFeature_cont'])}")
                feature_data['accFeature_cont'] = np.concatenate((feature_data['accFeature_cont'], accFeat), axis=0)
                feature_data['forceFeature_cont'] = np.concatenate((feature_data['forceFeature_cont'], forceFeat), axis=0)
                feature_data['trialLabel_cont'] = np.append(feature_data['trialLabel_cont'], trialLabel + cont_count)
                feature_data['subTrialLabel_cont'] = np.append(feature_data['subTrialLabel_cont'], activityGestureLabels)
            cont_count += 1

print(f"Final shape of emg disc: {np.shape(feature_data['emgFeature_disc'])}, emg cont: {np.shape(feature_data['emgFeature_cont'])}")

In [ ]:
feature_file_name = feat_path + 'daily_activities_S' + f'{subject:03}' + '_features.mat'
sio.savemat(feature_file_name, feature_data)
print(f"Data saved at\n{feature_file_name}")